In [2]:
import nltk
import matplotlib.pyplot as plt
import numpy
from nltk.corpus import twitter_samples
import random
import numpy as np
import math as m

nltk.download('twitter_samples')
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\Adi\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Adi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
# use predefined function ‘fileids()’ to see the content
twitter_samples.fileids()
# select the set of positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [4]:
# print positive in greeen
print('\033[92m' + all_positive_tweets[random.randint(0,5000)])

# print negative in red
print('\033[91m' + all_negative_tweets[random.randint(0,5000)])

Great to have him as part of the awesome #Dominion family :) https://t.co/sLmo2Voxzx
i’m so sad i want my hair colour back now :(


In [49]:

from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem import PorterStemmer        # module for stemming
from nltk.tokenize import TweetTokenizer   # module for tokenizing strings
import re
import string                              # for string operations

stop_words = stopwords.words('english')
punctuation = string.punctuation
tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)

tweets = all_positive_tweets + all_negative_tweets ## Concatenate the lists. 
labels = np.append(np.ones((len(all_positive_tweets),1)), np.zeros((len(all_negative_tweets),1)), axis = 0)

train_pos  = all_positive_tweets[:4000]
train_neg  = all_negative_tweets[:4000]
train_x = train_pos + train_neg 
train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)), axis=0).reshape(-1)


test_pos  = all_positive_tweets[1000:]
test_neg  = all_negative_tweets[1000:]
test_x = test_pos + test_neg
test_y = np.append(np.ones(len(test_pos)),np.zeros(len(test_neg)), axis=0)



In [6]:
print(train_y)

[1. 1. 1. ... 0. 0. 0.]


In [65]:
def preprocess(tweet) :
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = re.sub(r'https?://[^\s\n\r]+', '', tweet)
    tweet = re.sub(r'#', '', tweet)
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
    cleaned_stemmed_tweet = []
    for token in tweet_tokens:
        if token not in punctuation :
            cleaned_stemmed_tweet.append(token)
    return cleaned_stemmed_tweet




In [69]:


def buildFrequencyTable(features_x,labels_list):
    dictionary = {}
    sum_pos = 0
    sum_neg = 0
    for i in range(len(features_x)) :
        tokens = preprocess(features_x[i])
        for token in tokens:
            if token not in dictionary:
                if labels_list[i] == 1:
                    dictionary[token] = (1,0)
                    sum_pos+=1
                else:
                    dictionary[token] = (0,1)
                    sum_neg+=1
            else:
                if labels_list[i] == 1:
                    dictionary[token] = (dictionary[token][0]+1,dictionary[token][1])
                    sum_pos+=1
                else:
                    dictionary[token] = (dictionary[token][0], dictionary[token][1]+1)
                    sum_neg+=1
    return dictionary,sum_pos,sum_neg
                


In [70]:
def laplace_smoothing(dictionary,sum_pos, sum_neg, alpha):
    for key in dictionary:
        dictionary[key] = (dictionary[key][0]+alpha)/(sum_pos+len(dictionary)) , (dictionary[key][1]+alpha)/(sum_neg+len(dictionary))
    return dictionary

In [74]:
x_temp = ["I am happy because I am learning NLP I am happy, not sad","I am sad, I am not learning NLP I am sad,not happy"]
temp_labels = [1,0]
dictionary,sum_pos,sum_neg = buildFrequencyTable(x_temp,temp_labels)
print(dictionary)
dictionary = laplace_smoothing(dictionary,sum_pos,sum_neg,1)
print()
pos_test_sum = 0
neg_test_sum = 0
for key in dictionary:
    pos_test_sum+=dictionary[key][0]
    neg_test_sum+=dictionary[key][1]
print(f'{pos_test_sum} , {neg_test_sum}')

{'i': (3, 3), 'am': (3, 3), 'happy': (2, 1), 'because': (1, 0), 'learning': (1, 1), 'nlp': (1, 1), 'not': (1, 2), 'sad': (1, 2)}

0.9999999999999999 , 0.9999999999999998


Epoch 0, Loss: 0.6931471805599454
[  0.           7.67132438 -10.83586812]
Epoch 100, Loss: 0.05380451807778632
[ 3.60093532e-03  8.12504401e+00 -8.73426842e+00]
Epoch 200, Loss: 0.048563505419131035
[ 5.40354262e-03  7.37899846e+00 -7.68161702e+00]
Epoch 300, Loss: 0.04835419074449679
[ 0.00690303  6.49537431 -6.76040971]
Epoch 400, Loss: 0.04824862446026552
[ 0.00839992  5.611057   -5.83987276]
Epoch 500, Loss: 0.04815211106119109
[ 0.00989522  4.72672811 -4.91935457]
Epoch 600, Loss: 0.04785769137222982
[ 0.01138915  3.84240224 -3.99884635]
Epoch 700, Loss: 0.04755400561889181
[ 0.01288289  2.95809305 -3.07834452]
Epoch 800, Loss: 0.047261201802485886
[ 0.01437939  2.07381784 -2.15785137]
Epoch 900, Loss: 0.046023274571399
[ 0.01588388  1.18960283 -1.23739229]
Epoch 1000, Loss: 0.04247813627540391
[ 0.01741819  0.30526287 -0.31754746]


0.995125


[ 0.01741819  0.30526287 -0.31754746]
